# Importing Libraries

In [10]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [0]:
import warnings
warnings.filterwarnings('ignore')

!pip install fuzzywuzzy

# importing libraries
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests
from fuzzywuzzy import fuzz

import nltk
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.util import ngrams
from collections import Counter
from nltk import tokenize
import re
import gensim
from gensim.models import Word2Vec
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from collections import Counter

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Loading Lexicons

In [11]:
# reading the different lexicons and defining negative words
negative_words = ["no", "not", "not any", "none", "never", "neither", "nor", "few", "hardly", "little", "scarcely", "rarely", "seldom", "isn't", "don't", "can't", "hadn't"]

xlsx_file = '/content/drive/My Drive/Sustainable-Peace-Project/BBCLexicon.xlsx'
dfs = pd.read_excel(xlsx_file)
dfs = dfs.drop(['Unnamed: 1','Unnamed: 2','Unnamed: 3'], axis=1)
dfs.columns = ['Peace_Lexicon']
#preprocessing the peace lexicon words 
dfs['Peace_Lexicon'] = dfs['Peace_Lexicon'].str.lower()
dfs['Peace_Lexicon'] = dfs['Peace_Lexicon'].str.replace("_", " ")
dfs['Peace_Lexicon'] = dfs['Peace_Lexicon'].str.strip()
peace_lexicon_list = list(dfs.Peace_Lexicon)

conflict_file = '/content/drive/My Drive/Sustainable-Peace-Project/NewsLexicon-Conflict.csv'
dfs = pd.read_csv(conflict_file)
dfs.columns = ['Conflict_Lexicon']
#preprocessing the conflict lexicon words 
dfs['Conflict_Lexicon'] = dfs['Conflict_Lexicon'].str.lower()
dfs['Conflict_Lexicon'] = dfs['Conflict_Lexicon'].str.replace("_", " ")
dfs['Conflict_Lexicon'] = dfs['Conflict_Lexicon'].str.strip()
conflict_lexicon_list = list(dfs.Conflict_Lexicon)

nonPeace_file = '/content/drive/My Drive/Sustainable-Peace-Project/non_peaceful.csv'
dfs = pd.read_csv(nonPeace_file)
dfs.columns = ['NonPeaceful_Lexicon']
#preprocessing the non peace lexicon words
dfs['NonPeaceful_Lexicon'] = dfs['NonPeaceful_Lexicon'].str.lower()
dfs['NonPeaceful_Lexicon'] = dfs['NonPeaceful_Lexicon'].str.replace("_", " ")
dfs['NonPeaceful_Lexicon'] = dfs['NonPeaceful_Lexicon'].str.strip()
nonPeace_lexicon_list = list(dfs.NonPeaceful_Lexicon)

FileNotFoundError: ignored

In [0]:
# preprocessing the lexicons further by removing stem words
peace_processed = []
nonPeace_processed = []
conflict_processed = []
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
for i in peace_lexicon_list:
  input_str = word_tokenize(i)
  input_str_1 = [x for x in input_str if not x in ENGLISH_STOP_WORDS]
  peace_processed.append(' '.join(input_str_1))
for i in nonPeace_lexicon_list:
  input_str = word_tokenize(i)
  input_str_1 = [x for x in input_str if not x in ENGLISH_STOP_WORDS]
  nonPeace_processed.append(' '.join(input_str_1))
for i in conflict_lexicon_list:
  input_str = word_tokenize(i)
  input_str_1 = [x for x in input_str if not x in ENGLISH_STOP_WORDS]
  conflict_processed.append(' '.join(input_str_1))

# Defining functions

In [0]:
# building the gensim model
def build_model(text, lexicon):
  # iterate through each sentence in the file 

  text.append(lexicon)

  model = gensim.models.Word2Vec(text, min_count = 1,  
                                size = 100, window = 5, sg = 1)
  return model

# scraping data from bbc.com
def bbc_news(url):    
    response = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.120 Safari/537.36'})
    soup = BeautifulSoup(response.content, "lxml")
    article = soup.findAll('p') 
    article = article[12:27]
    news_article = ""
    for a in article:
        temp = a.get_text()
        news_article = news_article + " " + temp
    return news_article

# preprocessing with the text
def preprocess_text(text):
    text = re.sub('[!@#$,"()'':]', '', text.lower())
    sentences = tokenize.sent_tokenize(text)
    words = []
    temp = []
    for x in sentences:
        temp = x.split(' ')
        temp = [x for x in temp if not x in ENGLISH_STOP_WORDS] # removing stop words
        temp = [x.strip('.') for x in temp] 
        words.append(temp)
        temp = []
    return(words)

# functions for finding the peace / non-peace / conflict scores - 
def peace_score(words, lexicon):
    peace_list = []
    t = []
    score = 0
    for x in words:
        for y in x:
            temp = compare_terms(y, lexicon)
            if y in negative_words and flag == 0:
                # print(y)
                flag = 1
            elif temp != None and flag == 0:
                # print(y + ', ' + temp)
                t.append(y)
                t.append(temp)
                peace_list.append(t)
                t = []
                score = score + 1
            elif temp != None and flag == 1:
                # print(y + ', ' + temp)
                t.append(y)
                t.append(temp)
                peace_list.append(t)
                t = []
                score = score - 1
                flag = 0
        flag = 0
    return peace_list, score

# function to compare terms
def compare_terms(term, lexicon):
    for i in lexicon:
      term = term.strip(' ')
      term = term.strip('.')
      # term = term.strip('\'s')
      if term != '' and fuzz.ratio(term, i) > 85 and model.similarity(term, i) > 0.1:
        return i
# scraping data from the Google News API
# this function will find BBC news articles from a particular country in English Language
def google_news_articles(country):
    url = ('https://newsapi.org/v2/everything?'
           'domains=bbc.com&'
           'q=' + country + '&'
           'language=en&'
           'apiKey=78d0091a1deb4db6a93daa7fcafe5985')
    response = requests.get(url)
    #print(response.json())
    news = response.json()['articles']
    desc = []
    for i in range(len(news)):
        desc.append(news[i]['description'])
    url = []
    for i in range(len(news)):
        url.append(news[i]['url'])
    return url, desc

# Loading news articles

In [0]:
canada_file = '/content/drive/My Drive/Sustainable-Peace-Project/canada.csv'
dfs = pd.read_csv(canada_file)
dfs = dfs.drop(columns = 'Unnamed: 0')
dfs.columns = ['Text']
canada_news = list(dfs.Text)
canada_news_total = ' '.join(canada_news)

nz_file = '/content/drive/My Drive/Sustainable-Peace-Project/nz.csv'
dfs = pd.read_csv(nz_file)
dfs = dfs.drop(columns = 'Unnamed: 0')
dfs.columns = ['Text']
nz_news = list(dfs.Text)
nz_news_total = ' '.join(nz_news)

turkey_file = '/content/drive/My Drive/Sustainable-Peace-Project/turkey.csv'
dfs = pd.read_csv(turkey_file)
dfs = dfs.drop(columns = 'Unnamed: 0')
dfs.columns = ['Text']
turkey_news = list(dfs.Text)
turkey_news_total = ' '.join(turkey_news)

venezuela_file = '/content/drive/My Drive/Sustainable-Peace-Project/venezuela.csv'
dfs = pd.read_csv(venezuela_file)
dfs = dfs.drop(columns = 'Unnamed: 0')
dfs.columns = ['Text']
venezuela_news = list(dfs.Text)
venezuela_news_total = ' '.join(venezuela_news)

In [0]:
# preprocessing the text - removing punctuation marks and removing stop words
canada_words = preprocess_text(canada_news_total)
nz_words = preprocess_text(nz_news_total)
turkey_words = preprocess_text(turkey_news_total)
venezuela_words = preprocess_text(venezuela_news_total)

# Finding terms from our lexicon

In [0]:
model = build_model(canada_words, peace_processed)
canada_words = canada_words[:-1]
cterms, cscore = peace_score(canada_words, peace_processed)
print(cscore)

model = build_model(nz_words, peace_processed)
nz_words = nz_words[:-1]
nzterms, nzscore = peace_score(nz_words, peace_processed)
print(nzscore)

model = build_model(turkey_words, peace_processed)
turkey_words = turkey_words[:-1]
tterms, tscore = peace_score(turkey_words, peace_processed)
print(tscore)

model = build_model(venezuela_words, peace_processed)
venezuela_words = venezuela_words[:-1]
vterms, vscore = peace_score(venezuela_words, peace_processed)
print(vscore)

81
70
85
22


In [0]:
cterms1 = [x[1] for x in cterms]
canada_count = Counter(cterms1)
canada_count.most_common()

[('positive', 10),
 ('travel', 6),
 ('family', 4),
 ('rights', 4),
 ('help', 4),
 ('resources', 4),
 ('spiritual', 3),
 ('food', 3),
 ('culture', 3),
 ('development', 3),
 ('local', 2),
 ('sports', 2),
 ('tourism', 2),
 ('conciliation', 2),
 ('media', 2),
 ('community', 2),
 ('commonwealth', 2),
 ('healthcare', 2),
 ('liberal', 2),
 ('enforced', 1),
 ('living', 1),
 ('heritage', 1),
 ('live', 1),
 ('understanding', 1),
 ('mediation', 1),
 ('preserve', 1),
 ('trading', 1),
 ('empowerment', 1),
 ('thought', 1),
 ('stable', 1),
 ('dancing', 1),
 ('cooperation', 1),
 ('togetherness', 1),
 ('exchange', 1),
 ('concert', 1),
 ('dialogue', 1),
 ('cognition', 1),
 ('growth', 1),
 ('gather', 1),
 ('combine', 1)]

In [0]:
tterms1 = [x[1] for x in tterms]
turkey_count = Counter(tterms1)
turkey_count.most_common()

[('help', 11),
 ('local', 9),
 ('religion', 6),
 ('rights', 5),
 ('humanitarian', 4),
 ('agreement', 4),
 ('regime', 3),
 ('media', 3),
 ('cooperation', 3),
 ('solidarity', 3),
 ('happy', 3),
 ('living', 3),
 ('union', 2),
 ('internet', 2),
 ('enforced', 2),
 ('calm', 2),
 ('travel', 2),
 ('respect', 2),
 ('education', 2),
 ('moral', 1),
 ('character', 1),
 ('training', 1),
 ('participation', 1),
 ('coaction', 1),
 ('resources', 1),
 ('compassion', 1),
 ('community', 1),
 ('family', 1),
 ('exchange', 1),
 ('live', 1),
 ('food', 1),
 ('scientific', 1),
 ('responsibility', 1)]